In [1]:
!pip install dspy-ai openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 3.7 MB/s eta 0:00:00
  Using cached xxhash-3.5.0-cp311-cp311-macosx_11_0_arm64.whl.metadata (12 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.9/73.9 kB 6.3 MB/s eta 0:00:00
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached multiprocess-0.70.16-py311-none-any.whl.metadata (7.2 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.0/259.0 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 24.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.6 MB/s eta 0:00:00
Using cached xxhash-3.5.0-cp311-cp311-macosx_11_0_arm64.whl (30 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.8/471.8 kB 19.0 MB/s eta 0

In [6]:
import dspy

# Option 1: OpenAI
# lm = dspy.OpenAI(model="gpt-4")
# dspy.settings.configure(lm=lm)

# Option 2: Ollama (for local models)
lm = dspy.LM('ollama/llama3.1', api_base='http://localhost:11434', api_key='')
dspy.configure(lm=lm)

In [11]:
class QA(dspy.Signature):
    """Answer questions with short factoid answers."""
    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 5 words")

# Initialize CoT predictor
cot_predictor = dspy.ChainOfThought(QA)

# Test it
response = cot_predictor(question="What is the capital of Canada?")
print("Answer:", response.answer)
print("Reasoning:", response.reasoning)  # Shows CoT steps

Answer: Ottawa
Reasoning: The capital of Canada can be determined by recalling basic geographical knowledge.


In [15]:
import dspy
import json
from typing import List, Dict, Optional

# Define the signature for invoice parsing
class InvoiceParser(dspy.Signature):
    """Extract structured invoice data from raw OCR text and return as JSON."""
    ocr_text = dspy.InputField(desc="Raw OCR text from invoice")
    structured_json = dspy.OutputField(desc="Structured invoice data in JSON format")

# Initialize the predictor (we'll use ChainOfThought for better reasoning)
invoice_predictor = dspy.ChainOfThought(InvoiceParser)

# Load OCR text from file
with open("../ocr_parser/ocr_output.json", "r") as f:
    ocr_data = json.load(f)

# Convert OCR lines to raw text string
def extract_raw_text(ocr_data):
    raw_lines = []
    for block in ocr_data['pages'][0]['blocks']:
        for line in block.get('lines', []):
            words = [word['value'] for word in line.get('words', [])]
            if words:
                raw_lines.append(" ".join(words))
    return "\n".join(raw_lines)

ocr_text = extract_raw_text(ocr_data)

# Define the system prompt as part of the DSPy program
class InvoiceExtractionProgram(dspy.Module):
    def __init__(self):
        super().__init__()
        self.extractor = dspy.ChainOfThought(InvoiceParser)
    
    def forward(self, ocr_text):
        # Prepare the prompt with all requirements
        augmented_prompt = f"""
        Extract structured invoice data from the following raw OCR text:

        {ocr_text}

        Ensure that:
        - All line items are included
        - Final tax amounts (CGST, SGST, IGST) and grand total are captured
        - Bank details (bank name, account number, IFSC, UPI ID if present) are extracted

        Return only valid JSON with the following structure:
        {{
          "vendor": {{
            "name": ...,
            "address": ...,
            "contact": [...],
            "gstin": ...
          }},
          "invoice": {{
            "number": ...,
            "date": ...
          }},
          "bill_to": {{
            "name": ...,
            "address": ...
          }},
          "ship_to": {{
            "name": ...,
            "address": ...
          }},
          "bank_details": {{
            "bank_name": ...,
            "account_number": ...,
            "ifsc": ...,
            "upi_id": ...
          }},
          "line_items": [
            {{
              "description": ...,
              "hsn": ...,
              "uom": ...,
              "quantity": ...,
              "rate": ...,
              "taxable_value": ...,
              "cgst": {{
                "rate": ...,
                "amount": ...
              }},
              "sgst": {{
                "rate": ...,
                "amount": ...
              }}
            }}
          ],
          "total": {{
            "subtotal": ...,
            "cgst_total": ...,
            "sgst_total": ...,
            "igst_total": ...,
            "grand_total": ...
          }}
        }}
        """
        
        # Get the prediction
        response = self.extractor(ocr_text=augmented_prompt)
        return response

# Initialize the program
program = InvoiceExtractionProgram()

# Call the program
response = program(ocr_text)

# Process the response
print("Model response received. Parsing JSON...")
print("Answer:", response.structured_json)
print("Reasoning:", response.reasoning)  # Shows CoT steps

# Try to parse JSON and save to file
try:
    # First attempt to parse the JSON
    structured_data = json.loads(response.structured_json)
    
    # Save to file
    with open("response.json", "w", encoding="utf-8") as f:
        json.dump(structured_data, f, ensure_ascii=False, indent=2)
    print("JSON successfully saved to response.json")
    
except json.JSONDecodeError as e:
    print(f"Failed to parse JSON: {e}")
    print("Raw model response:")
    print(response.structured_json)
    
    # Save the raw response anyway for debugging
    with open("response_raw.txt", "w", encoding="utf-8") as f:
        f.write(response.structured_json)
    print("Saved raw response to response_raw.txt")

Model response received. Parsing JSON...
Answer: {
    "vendor": {
        "name": "RAJ SUPER WHOLESALE BAZAR",
        "address": "45,AMBA PRASAD TIWARI MARG, DAULATGANJ UAIN-MP-456001",
        "contact": ["0734-4060723", "9993736333"],
        "gstin": "23CTOPS449201ZX WRFAMT de"
    },
    "invoice": {
        "number": "CN3- 2254",
        "date": "27-Feb-2019"
    },
    "bill_to": {
        "name": "RAJ DATA PROCESSORS",
        "address": ""
    },
    "ship_to": {
        "name": "RAJ DATA PROCESSORS",
        "address": "Add 45, DAULATGANJ, UJJAIN"
    },
    "bank_details": {
        "bank_name": "BANK OF INDIA",
        "account_number": "9100123456456",
        "ifsc": "BKID00001901",
        "upi_id": ""
    },
    "line_items": [
        {
            "description": "SWADIST SOYA OIL ILTR (POUCH)",
            "hsn": "23- Madhya Pradesh",
            "uom": "",
            "quantity": 10,
            "rate": 78.10,
            "taxable_value": 780.95,
            "cgst":